## Predicting Future Trends in Airline Customer Sentiment Using Sentiment Analysis and incorporating Time Series Forecasting

#### AUTHORS

* Jeremiah Waiguru
* Mercy Kiragu
* Paul Ngatia
* Winfred Kinya

## 1.0 Project Overview

This project aims to analyze and predict customer sentiment towards various airlines by leveraging a dataset of customer feedback. The project will apply natural language processing (NLP) techniques to classify feedback as positive, negative, or neutral and use time series analysis to forecast future sentiment trends. These insights will help airlines enhance customer satisfaction, tailor their services, and address issues proactively.

## 1.1 Business Understanding



## 1.2 Problem Statement

The airline industry is currently facing a notable decrease in customer satisfaction, leading to unfavorable brand perception and diminished customer loyalty. This decline in satisfaction can be attributed to several factors, including flight delays, inadequate customer service, mishandling of luggage, and other operational inefficiencies. As a result, addressing these customer concerns and enhancing the overall brand perception has become a crucial focus for airlines.

## 1.3 Objectives

### Primary objective

To analyze and predict customer sentiment towards various airlines using sentiment analysis and time series forecasting, providing actionable insights to improve customer satisfaction and operational strategies.

### Specific objectives

1.	Implement a real-time monitoring system to continuously capture and process tweets related to airlines from Twitter.
2.	Implement and compare various NLP models (e.g., Logistic Regression, Naive Bayes, BERT, LSTM) for sentiment classification.
3.	Develop time series forecasting models to predict future trends in customer sentiment. 
4.	Generate actionable insights and recommendations based on sentiment analysis to improve customer satisfaction, address pain points, and enhance overall brand reputation. 
5.	Establish an effective response and engagement strategy to manage negative sentiment, address customer complaints, and foster positive customer experiences.


## 2.0  Data Understanding

Our dataset was publicly sourced from crowdflower website and is made up of Twitter users' tweets and retweets. The dataset has 14,640 rows and 20 columns. This Twitter data was collected from February 2015 and contributors were engaged in classifying tweets into categories of positive, negative, and neutral sentiments.